In [1]:
%matplotlib inline

In [2]:
# set dependencies
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from datetime import datetime
from datetime import date
from pprint import pprint
import seaborn as sns
import warnings
import itertools
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [3]:
# EIA API key
api_key = '9bb1b20524b7bea5b851a57f69e910f4'

# config info to call EIA website
base_url = 'http://api.eia.gov/series/?'
brent = 'PET.RBRTE.M'
colombian_landed = 'PET.ICO0000008.M'

# partial query url
brent_query_url = f'{base_url}api_key={api_key}&series_id={brent}'
colombian_landed_query_url = f'{base_url}api_key={api_key}&series_id={colombian_landed}'

In [4]:
# query EIA to get Brent prices (monthly)
brent_price = requests.get(brent_query_url)
brent_price_json = brent_price.json()
# pprint(brent_price_json)

# query EIA to get Brent prices (monthly)
colombian_landed = requests.get(colombian_landed_query_url)
colombian_landed_json = colombian_landed.json()
# pprint(colombian_landed_json)

In [8]:
# convert brent dictionary to pandas df
brent_df = pd.DataFrame(brent_price_json['series'][0]['data'], columns = ['Date', 'Dollars/bbl'])
brent_df['Date'] = brent_df['Date'].apply(lambda x : str(x[0:4]) + '-' + str(x[4:6]) + '-01')
brent_df['Date'] = brent_df['Date'].astype('datetime64[ns]')

# convert colombian landed dictionary to pandas df
colombian_landed_df = pd.DataFrame(colombian_landed_json['series'][0]['data'], columns = ['Date', 'Dollars/bbl'])
colombian_landed_df['Date'] = colombian_landed_df['Date'].apply(lambda x : str(x[0:4]) + '-' + str(x[4:6]) + '-01')
colombian_landed_df['Date'] = colombian_landed_df['Date'].astype('datetime64[ns]')

# combine brent and colombian landed dataframes together, merge on date
brent_colombia = pd.merge(brent_df, colombian_landed_df, on='Date', how ='inner')
# brent_colombia.head()

# rename colomns to brent and colombian landed
brent_colombia_renamed = brent_colombia.rename(columns={'Dollars/bbl_x': 'Colombian Landed', 'Dollars/bbl_y': 'Brent'})
# brent_colombia_renamed.head()

# create column to hold difference in Brent vs Colombian Landed $
delta = (brent_colombia_renamed['Colombian Landed'] - brent_colombia_renamed['Brent'])
brent_colombia_renamed['Delta'] = delta
# brent_colombia_renamed.head()

# create month and year columns and set date as index and rename df
year = brent_colombia_renamed['Date'].dt.year
brent_colombia_renamed['Year'] = year

month = brent_colombia_renamed['Date'].dt.month
brent_colombia_renamed['Month'] = month
brent_colanded = brent_colombia_renamed.set_index('Date')
brent_colanded = brent_colanded.resample('M').mean()
# brent_colanded.head()
brent_colanded.describe()

,Colombian Landed,Brent,Delta,Year,Month
count,275.000000,274.000000,274.000000,275.000000,275.000000
mean,56.391855,53.975620,2.456058,2006.960000,6.480000
std,33.245054,30.691129,3.823765,6.624044,3.448601
min,9.820000,10.050000,-5.130000,1996.000000,1.000000
25%,25.800000,27.097500,-0.700000,2001.000000,3.500000
50%,51.700000,48.335000,1.930000,2007.000000,6.000000
75%,77.010000,73.140000,5.032500,2013.000000,9.000000
max,132.720000,127.350000,15.620000,2018.000000,12.000000


In [17]:
# Time to grab route data for comparison 

# identify file and create file path
file = "C:\\Users\\maque\\Dropbox\\Rice Data analytics classes\\RICEHOU201811DATA2\\ETL_project\\tanker_routes.xlsx"

route = 'Baltic Dirty Tanker Route TD9 70000mt Caribs to US Gulf'

# read file
df = pd.read_excel(file)
# df.head(1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 27 columns):
Date                                                                            700 non-null datetime64[ns]
Baltic Dirty Tanker Route TD3C 270000mt ME Gulf to China                        700 non-null float64
Baltic Dirty Tanker Route TD9 70000mt Caribs to US Gulf                         700 non-null float64
Baltic Dirty Tanker Route TD20 130000mt W Africa to UK Cont                     700 non-null float64
Baltic Dirty Tanker Route TD21 Caribbean to US Gulf (50000mt Fuel Oil)          700 non-null float64
Argus Dirty Caribbean - Singapore 270kt VLCC (lumpsum) Prmpt                    700 non-null int64
Argus Freight dirty USGC/Caribbean - Singapore 130kt (lumpsum) Prmpt            700 non-null int64
Argus Freight dirty Caribbean - China 270kt (lumpsum) Prmpt                     700 non-null int64
Baltic Clean Tanker Route TC14 38000mt CPP/UNL/Diesel U.S. Gulf to Continent    700 no

In [18]:
# let's drop some columns
df.set_index('Date', inplace=True)
cols = ['Add #1', 'Add #2', 'Add #3', 
        'Add #4', 'Add #5', 'Add #6', 
        'Add #7', 'Add #8', 'Add #9', 'Add #10']
df.drop(cols, axis=1, inplace=True)
df = df.sort_values('Date')
df.isnull().sum()

Baltic Dirty Tanker Route TD3C 270000mt ME Gulf to China                        0
Baltic Dirty Tanker Route TD9 70000mt Caribs to US Gulf                         0
Baltic Dirty Tanker Route TD20 130000mt W Africa to UK Cont                     0
Baltic Dirty Tanker Route TD21 Caribbean to US Gulf (50000mt Fuel Oil)          0
Argus Dirty Caribbean - Singapore 270kt VLCC (lumpsum) Prmpt                    0
Argus Freight dirty USGC/Caribbean - Singapore 130kt (lumpsum) Prmpt            0
Argus Freight dirty Caribbean - China 270kt (lumpsum) Prmpt                     0
Baltic Clean Tanker Route TC14 38000mt CPP/UNL/Diesel U.S. Gulf to Continent    0
Argus Clean USGC - Chile 38kt (lumpsum) Prmpt                                   0
Argus Freight clean Houston - east coast Mexico 38kt (lumpsum) Prmpt            0
Argus Freight clean Houston - Pozos 38kt (lumpsum) Prmpt                        0
Argus Freight clean USGC - Argentina/Brazil 38kt WS Prmpt                       0
Day of Month    

In [19]:
# group data by date and set date as index
df = df.groupby('Date')[route].sum().reset_index()

# create month and year columns
year = df['Date'].dt.year
df['Year'] = year
month = df['Date'].dt.month
df['Month'] = month
# df.head()

# set date as index
df = df.set_index('Date')
route_df = df.resample('M').mean()
# df.head()

# combine brent and colombian landed dataframes together, merge on date
crude_vs_freight = pd.merge(brent_colanded, route_df, on=['Date'], how ='left')
# crude_vs_freight.head()

# clean df by drop nulls and renaming columns
cleaned_crude_freight = crude_vs_freight.dropna()
cleaned_crude_freight = cleaned_crude_freight.drop(['Year_y', 'Month_y'], axis=1)
cleaned_crude_freight = cleaned_crude_freight.rename(columns={'Year_x': 'Year', 'Month_x': 'Month'})
cleaned_again_crude_freight = cleaned_crude_freight[['Year', 'Month', 'Brent', 'Colombian Landed', 'Delta', route]]
cleaned_again_crude_freight

,Year,Month,Brent,Colombian Landed,Delta,Baltic Dirty Tanker Route TD9 70000mt Caribs to US Gulf
Date,,,,,,
2016-06-30,2016,6,43.88,48.25,4.37,87.128333
2016-07-31,2016,7,40.90,44.95,4.05,79.904286
2016-08-31,2016,8,40.78,45.84,5.06,80.767826
2016-09-30,2016,9,43.43,46.57,3.14,91.790909
2016-10-31,2016,10,43.44,49.52,6.08,95.306667
2016-11-30,2016,11,42.97,44.73,1.76,123.100909
2016-12-31,2016,12,48.83,53.31,4.48,138.267727
2017-01-31,2017,1,49.17,54.58,5.41,159.763636
2017-02-28,2017,2,49.66,54.87,5.21,128.596500


In [20]:
# send data to mySQL
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy import create_engine

In [28]:
# Create engine to connect to database

dbuser = 'root'
dbpassword = '8891lobtuF!'
dbhost = 'localhost'
dbport = '3306'
dbname= 'oil_db'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")
cleaned_again_crude_freight.to_sql(name='brent_vs_landed', con=engine, if_exists = 'append', index=False)